In [ ]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os

In [ ]:
img=cv2.imread('D:/TSF/COLOR.png')

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(img)
print("Shape: {}".format(img.shape))

In [ ]:
grid_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(grid_RGB)

In [ ]:
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.figure(figsize=(20,8))
plt.imshow(gray_image, cmap='gray')

In [ ]:
resized_image = cv2.resize(grid_RGB, (1000, 500))
plt.figure(figsize=(20,8))
plt.imshow(resized_image)

In [ ]:
grid_HSV = cv2.cvtColor(grid_RGB,cv2.COLOR_RGB2HSV)

In [ ]:
lower = np.array([25,150,50])
upper = np.array([35,255,255])

In [ ]:
mask=cv2.inRange(grid_HSV,lower,upper)

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(mask)

In [ ]:
res = cv2.bitwise_and(grid_RGB, grid_RGB,mask=mask)

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(res)

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
image_path='D:/TSF/COLOR.png'
def get_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
def get_colors(image, number_of_colors, show_chart):
    
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    
    counts = Counter(labels)
    # sort to obtain the correct color percentage
    counts = dict(sorted(counts.items()))
    
    center_colors = clf.cluster_centers_
    
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if (show_chart):
        plt.figure(figsize = (8, 6))
        plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)
    
    return rgb_colors

In [ ]:
get_colors(img, 5, True)

In [ ]:
IMAGE_DIRECTORY = 'D:/TSF'
COLORS = {
    'GREEN': [0, 128, 0],
    'BLUE': [0, 0, 128],
    'RED': [128, 0, 0]
}
images = []

for file in os.listdir(IMAGE_DIRECTORY):
    if not file.startswith('.'):
        images.append(get_image(os.path.join(IMAGE_DIRECTORY, file)))

In [ ]:
plt.figure(figsize=(20, 10))
for i in range(len(images)):
    plt.subplot(1, len(images), i+1)
    plt.imshow(images[i])

In [ ]:
def match_image_by_color(image, color, threshold = 60, number_of_colors = 10): 
    
    image_colors = get_colors(image, number_of_colors, False)
    selected_color = rgb2lab(np.uint8(np.asarray([[color]])))

    select_image = False
    for i in range(number_of_colors):
        curr_color = rgb2lab(np.uint8(np.asarray([[image_colors[i]]])))
        diff = deltaE_cie76(selected_color, curr_color)
        if (diff < threshold):
            select_image = True
    
    return select_image

In [ ]:
def show_selected_images(images, color, threshold, colors_to_match):
    index = 1
    
    for i in range(len(images)):
        selected = match_image_by_color(images[i],
                                        color,
                                        threshold,
                                        colors_to_match)
        if (selected):
            plt.subplot(1, 5, index)
            plt.imshow(images[i])
            index += 1

In [ ]:
# Variable selected_color can be of COLORS['GREEN'], COLORS['BLUE'] or COLORS['RED']
plt.figure(figsize = (20, 8))
show_selected_images(images, COLORS['GREEN'], 60, 5)